# Question 2 - Part A
In the job scheduling problem discussed in the class slides, assume that the daily workforce demand follows the same table as given in the slides.

Day 1 (Saturday): 17 employees  
Day 2 (Sunday): 13 employees  
Day 3 (Monday): 15 employees  
Day 4 (Tuesday): 19 employees  
Day 5 (Wednesday): 14 employees  
Day 6 (Thursday): 16 employees  
Day 7 (Friday): 11 employees  

However, this time, the company wants to hire two types of full-time workers: regular full-time workers and special full-time workers.

Each regular full-time worker must work 5 consecutive days and have 2 days off.
Each special full-time worker must work 6 days and have 1 day off.

The monthly salary of a regular full-time worker is c1 dollars.
The daily wage of a special full-time worker is c2 dollars.

Additionally, at most 75% of the total workforce can be regular full-time workers.

Formulate an optimization model that minimizes the total monthly wage cost while satisfying the daily workforce demand.

In [ ]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [25]:
import pyomo.environ as pyo


# Create model
model = pyo.ConcreteModel()

# Define the range for i (1 to 7)
model.I = pyo.RangeSet(1, 7)

# Define parameters c1 and c2
c1_value = 70
c2_value = 95

model.c1 = pyo.Param(initialize=c1_value, mutable=True)
model.c2 = pyo.Param(initialize=c2_value, mutable=True)

# Define decision variables x(i) and y(i)
model.x = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)

# Define the objective function
def obj_rule(model):
    return model.c1 * sum(model.x[i] for i in model.I) + 24 * model.c2 * sum(model.y[i] for i in model.I)

# Add the objective function to the model
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Define constraints
model.const1 = pyo.Constraint(
    expr=model.x[1] + model.x[7] + model.x[6] + model.x[5] + model.x[4] +
         model.y[1] + model.y[7] + model.y[6] + model.y[5] + model.y[4] + model.y[3] >= 17
)

model.const2 = pyo.Constraint(
    expr=model.x[2] + model.x[1] + model.x[7] + model.x[6] + model.x[5] +
         model.y[2] + model.y[1] + model.y[7] + model.y[6] + model.y[5] + model.y[4] >= 13
)

model.const3 = pyo.Constraint(
    expr=model.x[3] + model.x[2] + model.x[1] + model.x[7] + model.x[6] +
         model.y[3] + model.y[2] + model.y[1] + model.y[7] + model.y[6] + model.y[5] >= 15
)

model.const4 = pyo.Constraint(
    expr=model.x[4] + model.x[3] + model.x[2] + model.x[1] + model.x[7] +
         model.y[4] + model.y[3] + model.y[2] + model.y[1] + model.y[7] + model.y[6] >= 19
)

model.const5 = pyo.Constraint(
    expr=model.x[5] + model.x[4] + model.x[3] + model.x[2] + model.x[1] +
         model.y[5] + model.y[4] + model.y[3] + model.y[2] + model.y[1] + model.y[7] >= 14
)

model.const6 = pyo.Constraint(
    expr=model.x[6] + model.x[5] + model.x[4] + model.x[3] + model.x[2] +
         model.y[6] + model.y[5] + model.y[4] + model.y[3] + model.y[2] + model.y[1] >= 16
)

model.const7 = pyo.Constraint(
    expr=model.x[7] + model.x[6] + model.x[5] + model.x[4] + model.x[3] +
         model.y[7] + model.y[6] + model.y[5] + model.y[4] + model.y[3] + model.y[2] >= 11
)

def const8_rule(model):
    return sum(model.x[i] for i in model.I) <= (0.75 * sum(model.x[i] + model.y[i] for i in model.I))

# Add constraints to the model
model.const8 = pyo.Constraint(rule=const8_rule)

# Create a solver and solve the model
solver = pyo.SolverFactory('glpk')
solution = solver.solve(model, tee=False)

# Display the results
model.display()

# Print values of decision variables manually
# print("\n" + "Optimal values of decision variables:")
# for i in model.I:
#     print(f"x({i}) = {model.x[i].value}")
# for i in model.I:
#     print(f"y({i}) = {model.y[i].value}")


# Store the optimal objective function value for part B
minimum_cost = pyo.value(model.obj)


Model unknown

  Variables:
    x : Size=7, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   4.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   6.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          6 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
          7 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    y : Size=7, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :  None : False : False : NonNegat

# Question 2 - Part B
Suppose the company has set another goal as the second objective. It wants to choose an answer from all the optimal solutions of part (A) where the number of employees whose day off is Friday is maximized. What optimization model should be solved to achieve this goal?

In [24]:
import pyomo.environ as pyo

# Create model
model = pyo.ConcreteModel()

# Define the range for i (1 to 7)
model.I = pyo.RangeSet(1, 7)

# Define parameters c1 and c2
c1_value = 70
c2_value = 95

model.c1 = pyo.Param(initialize=c1_value, mutable=True)
model.c2 = pyo.Param(initialize=c2_value, mutable=True)

# Define decision variables x(i) and y(i)
model.x = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)

# Define the objective function (maximize x1 + x2 + y1)
def obj_rule(model):
    return model.x[1] + model.x[2] + model.y[1]

# Add the objective function to the model
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define constraints
model.const1 = pyo.Constraint(
    expr=model.x[1] + model.x[7] + model.x[6] + model.x[5] + model.x[4] +
         model.y[1] + model.y[7] + model.y[6] + model.y[5] + model.y[4] + model.y[3] >= 17
)

model.const2 = pyo.Constraint(
    expr=model.x[2] + model.x[1] + model.x[7] + model.x[6] + model.x[5] +
         model.y[2] + model.y[1] + model.y[7] + model.y[6] + model.y[5] + model.y[4] >= 13
)

model.const3 = pyo.Constraint(
    expr=model.x[3] + model.x[2] + model.x[1] + model.x[7] + model.x[6] +
         model.y[3] + model.y[2] + model.y[1] + model.y[7] + model.y[6] + model.y[5] >= 15
)

model.const4 = pyo.Constraint(
    expr=model.x[4] + model.x[3] + model.x[2] + model.x[1] + model.x[7] +
         model.y[4] + model.y[3] + model.y[2] + model.y[1] + model.y[7] + model.y[6] >= 19
)

model.const5 = pyo.Constraint(
    expr=model.x[5] + model.x[4] + model.x[3] + model.x[2] + model.x[1] +
         model.y[5] + model.y[4] + model.y[3] + model.y[2] + model.y[1] + model.y[7] >= 14
)

model.const6 = pyo.Constraint(
    expr=model.x[6] + model.x[5] + model.x[4] + model.x[3] + model.x[2] +
         model.y[6] + model.y[5] + model.y[4] + model.y[3] + model.y[2] + model.y[1] >= 16
)

model.const7 = pyo.Constraint(
    expr=model.x[7] + model.x[6] + model.x[5] + model.x[4] + model.x[3] +
         model.y[7] + model.y[6] + model.y[5] + model.y[4] + model.y[3] + model.y[2] >= 11
)

def const8_rule(model):
    return sum(model.x[i] for i in model.I) <= (0.75 * sum(model.x[i] + model.y[i] for i in model.I))

# Add constraint to the model
model.const8 = pyo.Constraint(rule=const8_rule)

def cost_constraint_rule(model):
    return model.c1 * sum(model.x[i] for i in model.I) + 24 * model.c2 * sum(model.y[i] for i in model.I) <= minimum_cost

model.cost_constraint = pyo.Constraint(rule=cost_constraint_rule)

# Create a solver and solve the model
solver = pyo.SolverFactory('glpk')
solution = solver.solve(model, tee=False)

# Display the results
model.display()

Model unknown

  Variables:
    x : Size=7, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   1.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   6.0 :  None : False : False : NonNegativeIntegers
          5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          6 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          7 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    y : Size=7, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   1.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :  None : False : False : NonNegat